In [15]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

import dateutil

DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)

In [2]:
#connect to UMLS
try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print
    #cnx.close()

In [3]:
#Define functions
def findICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        return (icd10[0])
    return ("Undef")

In [4]:
asthmaCodes = findICD10('Asthma')
findICD10('Asthma')

u'J45.9'

In [5]:
## FHIR patient data
import requests
import urllib2
import json

#base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
#patients = base_url+"/Patient"+"?_count=50"

#req = requests.get(patients)

def findPatients(code):
    try:
        response = urllib2.urlopen("http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+code+"")
    except:
        print "I can't find patients"
        exit()
    html = response.read()
    return (html)

In [6]:
## Find FHIR patients with asthma codes
for c in asthmaCodes:
    print findPatients(c)


	{
    "resourceType" : "Bundle",
    "id" : "84803D75-5056-8036-4F9DBD8F12BBCF4E",
    "meta" : {
        "count" : "50",
        "lastUpdated" : "2017-05-10T14:37:15Z"
    },
    "type" : "collection",
    "entry" : [
	
		{
			"fullurl" : "http://ictrweb.johnshopkins.edu/rest/synthetic/Condition/1745",
				"resourceType" : "Condition", 
				"id": "1745",
				"meta": {
				  "versionId": "2",
				  "lastUpdated": "2017-05-10T14:37:15Z"
				},
				"status": "final",
				"clinicalStatus": "active",
				"verificationStatus": "confirmed",
				"category": [
				{
				  "coding": [
					{
					  "system": "http://snomed.info/sct",
					  "code": "55607006",
					  "display": "Problem"
					},
					{
					  "system": "http://hl7.org/fhir/condition-category",
					  "code": "problem-list-item"
					}
				  ]
				}
				],
				"severity": {
				"coding": [
				  {
					"system": "http://snomed.info/sct",
					"code": "371924009",
					"display": "Moderate to severe"
				  }
				]
				},
				"c

In [7]:
## Given disease/condition term, get back ICD codes
def txt2icd(txt):
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": "Asthma",
              "VOCABULARY_ID": ["ICD9CM"]
    }
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    icd9arr=[]
    for d in data:
        icd9arr.append(d["CONCEPT_CODE"])
    return icd9arr
print txt2icd('asthma')

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


In [8]:
## Given disease name, get back symptoms (defined using MeSH terms) along with TFIDF scores
## Taken from https://www.nature.com/articles/ncomms5212
def disease2symptom(txt):
    s = filter(lambda x: txt.lower() in x[1].lower(), DISEASE2SYMPTOMS)
    return([(x[0], x[3]) for x in s])
disease = "Asthma"
symps = disease2symptom(disease)
print 'Found %s symptom MeSH terms for %s' % (len(symps), disease)

Found 206 symptom MeSH terms for Asthma


In [9]:
## Pull patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

patients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
##pprint.pprint (patients)

Initializing thread provenance @thread: MainThread


In [10]:
meds = {}
for x in patients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 9,
 u'0.3 ML Epinephrine 1 MG/ML Auto-Injector [Epipen]': 1,
 u'120 ACTUAT Fluticasone propionate 0.05 MG/ACTUAT Nasal Inhaler': 2,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ADVAIR HFA 230-21 MCG INHALER': 1,
 u'ALBUTEROL 90 MCG INHALER': 1,
 u'ALBUTEROL SULF HFA 90 MCG INH': 7,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 9,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 19,
 u'AMOXICILLIN 250 MG CHEWABLE TABLET': 1,
 u'AMOXICILLIN 875 MG TABLET': 1,
 u'AUGMENTIN 400-57 MG/5 ML': 1,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 11,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 3,
 u'Amitriptyline Hydrochloride 10 MG Oral Tablet': 1,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 4,
 u

In [11]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")   

[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [12]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos} 

Processing HYDROCORTISONE VAL 0.2% CREAM and found 1 annotations
Processing CLARITIN 5 MG/5 ML SYRUP and found 1 annotations
Processing CLINDAMYCIN 150 MG/ML INJECTION SOLUTION and found 4 annotations
Processing PROAIR HFA 90 MCG INHALER and found 1 annotations
Processing ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION and found 2 annotations
Processing Fexofenadine hydrochloride 30 MG Oral Tablet and found 5 annotations
Processing ADVAIR HFA 230-21 MCG INHALER and found 1 annotations
Processing Ofloxacin 3 MG/ML Ophthalmic Solution and found 4 annotations
Processing levocetirizine dihydrochloride 5 MG Oral Tablet and found 5 annotations
Processing DIAZEPAM 2 MG TABLET and found 2 annotations
Processing Clonidine Hydrochloride 0.1 MG Oral Tablet and found 5 annotations
Processing LIDEX 0.05% OINTMENT and found 0 annotations
Processing KETOCONAZOLE 2 % TOPICAL CREAM and found 3 annotations
Processing BACTROBAN 2% OINTMENT and found 1 annotations
Processing FOCALIN XR 5

Processing Triamcinolone Acetonide 0.001 MG/MG Topical Ointment and found 5 annotations
Processing Glucose 50 MG/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution and found 7 annotations
Processing SUPRAX 200 MG/5 ML SUSPENSION and found 1 annotations
Processing LORTAB 7.5-500 MG/15 ML ELIXIR and found 1 annotations
Processing lisdexamfetamine dimesylate 50 MG Oral Capsule and found 5 annotations
Processing lansoprazole 30 MG Delayed Release Oral Capsule and found 5 annotations
Processing TAMIFLU 12 MG/ML SUSPENSION and found 1 annotations
Processing Amoxicillin 500 MG Oral Capsule and found 4 annotations
Processing Azithromycin 40 MG/ML Oral Suspension and found 4 annotations
Processing CONCERTA ER 27 MG TABLET and found 1 annotations
Processing Cyproheptadine hydrochloride 4 MG Oral Tablet and found 5 annotations
Processing AMOXICILLIN 250 MG CHEWABLE TABLET and found 4 annotations
Processing moxifloxacin 5 MG/ML Ophthalmic Solution and found 4 annotations
Processing AUGMENTIN ES

In [17]:
## Get asthma patients
url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10=j45&_count=1000'
txt = urlopen(url).read()
stuff = json.loads(txt)
#pprint.pprint(stuff)

In [18]:
res = []
for p in stuff['entry']:
   d1 = p['assertedDate']
   id = p['subject']['reference'][8:]
   url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/'+id
   txt = urlopen(url).read()
   st = json.loads(txt)
   #pprint.pprint(st)
   d2 = st['birthDate']
   zip = st['address'][0]['postalCode']
   d1 = dateutil.parser.parse(d1)
   d2 = dateutil.parser.parse(d2)
   #print d1
   #print d2
   diff = d1 - d2
   #print diff
   url = 'https://www.zipcodeapi.com/rest/lPMf5jmnBdclCZWkQwlFPJO6HkolG4N1TzgZSDnuRAPtzLOqi957STdzeBVVFIWz/multi-info.json/'+zip+'/degrees'
   txt = urlopen(url).read()
   st = json.loads(txt)
   #pprint.pprint(st)
   lat = st[zip]['lat']
   lng = st[zip]['lng']
   #print lat
   #print lng
   #print diff.days
   t = (id,d2,d1,diff.days,zip,lat,lng)
   res.append(t)

pprint.pprint(res)

[(u'886787037',
  datetime.datetime(1961, 7, 12, 0, 0),
  datetime.datetime(1988, 11, 5, 0, 0),
  9978,
  u'20011',
  53.849437,
  -26.029474),
 (u'550750435',
  datetime.datetime(1936, 3, 9, 0, 0),
  datetime.datetime(2004, 12, 13, 0, 0),
  25116,
  u'20002',
  93.336028,
  -33.43896),
 (u'759651142',
  datetime.datetime(1974, 12, 9, 0, 0),
  datetime.datetime(1984, 10, 8, 0, 0),
  3591,
  u'20036',
  42.917747,
  -62.717679),
 (u'282146129',
  datetime.datetime(1981, 6, 28, 0, 0),
  datetime.datetime(1999, 3, 2, 0, 0),
  6456,
  u'20020',
  58.340971,
  -44.893532),
 (u'970346722',
  datetime.datetime(1960, 3, 14, 0, 0),
  datetime.datetime(2006, 6, 23, 0, 0),
  16902,
  u'20746',
  47.430764,
  -70.042941),
 (u'616022443',
  datetime.datetime(1990, 10, 1, 0, 0),
  datetime.datetime(1994, 2, 20, 0, 0),
  1238,
  u'20019',
  79.571405,
  -69.495194),
 (u'251317163',
  datetime.datetime(1976, 4, 8, 0, 0),
  datetime.datetime(1999, 2, 10, 0, 0),
  8343,
  u'20016',
  55.552728,
  -42.13

  datetime.datetime(1996, 6, 18, 0, 0),
  datetime.datetime(2003, 11, 3, 0, 0),
  2694,
  u'20743',
  83.00199,
  -44.807588),
 (u'134516989',
  datetime.datetime(1925, 7, 4, 0, 0),
  datetime.datetime(1993, 3, 26, 0, 0),
  24737,
  u'20816',
  71.042584,
  -59.931383),
 (u'498571852',
  datetime.datetime(1976, 12, 13, 0, 0),
  datetime.datetime(2013, 7, 11, 0, 0),
  13359,
  u'20910',
  40.721716,
  -55.264071),
 (u'942155677',
  datetime.datetime(1997, 8, 6, 0, 0),
  datetime.datetime(2010, 5, 4, 0, 0),
  4654,
  u'20016',
  71.595546,
  -38.12862),
 (u'376386479',
  datetime.datetime(1954, 2, 6, 0, 0),
  datetime.datetime(1961, 8, 22, 0, 0),
  2754,
  u'20019',
  42.902106,
  -69.495194),
 (u'425069555',
  datetime.datetime(1988, 7, 26, 0, 0),
  datetime.datetime(2008, 12, 10, 0, 0),
  7442,
  u'20815',
  88.25773,
  -77.079436),
 (u'975849373',
  datetime.datetime(1972, 11, 1, 0, 0),
  datetime.datetime(2002, 2, 5, 0, 0),
  10688,
  u'20020',
  59.486886,
  -54.633815),
 (u'1417566

  -76.422829),
 (u'522153108',
  datetime.datetime(1988, 2, 6, 0, 0),
  datetime.datetime(2015, 10, 18, 0, 0),
  10116,
  u'20007',
  40.060293,
  -55.30704),
 (u'454277162',
  datetime.datetime(1992, 3, 9, 0, 0),
  datetime.datetime(1997, 9, 3, 0, 0),
  2004,
  u'20019',
  52.642388,
  -27.096317),
 (u'649952557',
  datetime.datetime(1976, 2, 2, 0, 0),
  datetime.datetime(1983, 4, 3, 0, 0),
  2617,
  u'20032',
  69.200466,
  -54.66132),
 (u'453216113',
  datetime.datetime(1989, 8, 1, 0, 0),
  datetime.datetime(1996, 4, 8, 0, 0),
  2442,
  u'20910',
  88.277213,
  -65.004354),
 (u'586683517',
  datetime.datetime(1942, 9, 11, 0, 0),
  datetime.datetime(1951, 2, 6, 0, 0),
  3070,
  u'20020',
  67.508295,
  -24.267052),
 (u'346671834',
  datetime.datetime(1974, 9, 14, 0, 0),
  datetime.datetime(1983, 9, 14, 0, 0),
  3287,
  u'22314',
  40.525706,
  -77.056521),
 (u'659955891',
  datetime.datetime(1991, 5, 26, 0, 0),
  datetime.datetime(2005, 12, 8, 0, 0),
  5310,
  u'20019',
  62.382671,


In [22]:
def findAgeZipcode(stuff):
   res = []
   for p in stuff['entry']:
       d1 = p['assertedDate']
       id = p['subject']['reference'][8:]
       url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/'+id
       txt = urlopen(url).read()
       st = json.loads(txt)
       #pprint.pprint(st)
       d2 = st['birthDate']
       zip = st['address'][0]['postalCode']
       d1 = dateutil.parser.parse(d1)
       d2 = dateutil.parser.parse(d2)
       #print d1
       #print d2
       diff = d1 - d2
       #print diff
       url = 'https://www.zipcodeapi.com/rest/lPMf5jmnBdclCZWkQwlFPJO6HkolG4N1TzgZSDnuRAPtzLOqi957STdzeBVVFIWz/multi-info.json/'+zip+'/degrees'
       txt = urlopen(url).read()
       st = json.loads(txt)
       #pprint.pprint(st)
       lat = st[zip]['lat']
       lng = st[zip]['lng']
       #print lat
       #print lng
       #print diff.days
       t = (id,d2,d1,diff.days,zip,lat,lng)
       res.append(t)
   return(res)

print asthmaCodes

## Find FHIR patients with asthma codes
#for c in asthmaCodes:
   
   ## Get asthma patients
   #url = "http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+c+"&_count=1000"
   #txt = urlopen(url).read()
   #stuff = json.loads(txt)
   #print c
   #pprint.pprint(findAgeZipcode(stuff))

J45.9
